# Qualitative Analysis: Ideas for Content and Thematic Analysis Using Python

* Load Notebook into Chat
* View notebook directly by uploading to: [Jupyter Lap](https://jupyter.org/try-jupyter/lab)


If you want to run this code yourself:
* Install VS Code, Python
* Install dependencies
* change path / header references to match your data

## Background Information

### Overview of Content and Thematic Analysis

#### Content Analysis
* **Purpose**: To systematically analyze the content of qualitative data, often to quantify and categorize specific elements.

* **Approach**:
    * **Quantitative Focus**: Involves coding the text into categories or themes and often quantifies the frequency of these categories.
    * **Focus**: Concentrates on identifying and counting the occurrence of specific words, phrases, or themes within the text. It can be either qualitative or quantitative, but often involves quantifying textual elements to identify trends or patterns.
    * **Outcome**: Provides a structured overview of the content, highlighting how often specific elements appear and how they relate to each other.
    * **Process**:
        * Define categories or codes.
        * Systematically code the text according to predefined criteria.
        * Quantify the occurrence of categories.
        * Analyze patterns and trends in the data.

* **Application**: Useful for obtaining a systematic and objective overview of the content, identifying frequently occurring themes or elements, and analyzing trends across large volumes of text.

#### Thematic Analysis
* **Purpose**: To identify, analyze, and report patterns (themes) within qualitative data.

* **Approach**:

    * **Inductive or Deductive**: Themes can emerge from the data (inductive) or be predefined (deductive).
    * **Focus**: Concentrates on understanding the underlying meanings and patterns within the text. It involves coding the data into themes and sub-themes that capture significant patterns or ideas.
    * **Outcome**: Provides insights into how different themes relate to each other and to the overall research questions, often aiming to interpret and understand the context and meaning behind the data.
    * **Process**:
        * Familiarize with the data.
        * Generate initial codes.
        * Search for themes.
        * Review and refine themes.
        * Define and name themes.
        * Write up the analysis.

* **Application**: Useful for exploring complex patterns and meanings within data, and understanding how different themes contribute to the overall narrative.

#### Summary
Thematic Analysis focuses on identifying and interpreting themes and patterns within qualitative data to understand meanings and context.
Content Analysis involves systematically categorizing and quantifying elements of the text to identify trends and frequencies, often with a more structured and objective approach.
Both methods provide valuable insights but serve different purposes: thematic analysis is more exploratory and interpretive, while content analysis is more systematic and quantifiable.

### Tools Used in This Presentation

* [VS Code](https://code.visualstudio.com/)
    * [Jupyter Overview](https://jupyter.org)
        * [Jupyter in VS Code](https://code.visualstudio.com/docs/datascience/jupyter-notebooks)
        * [Markdown](https://www.markdownguide.org/basic-syntax)
    


* [Python](https://www.python.org/)
    * Packages
        * [Pandas](https://pandas.pydata.org/docs)
        * [NLTK](https://www.nltk.org)

### Additional Resources

#### Python

* [Getting Started with Python in VS Code](https://code.visualstudio.com/docs/python/python-tutorial)
* [How to set up Python on Visual Studio Code](https://www.youtube.com/watch?v=9o4gDQvVkLU) (YouTube)



* [How to install Python Packages in VS Code](https://www.youtube.com/watch?v=InRmKECJK3s) (YouTube)
* [Python Package Index (PyPI)](https://pypi.org/) (additional packages for Python to extend it functionality)

#### Qualitative Research
* [The Practical Guide to Qualitative Content Analysis](https://delvetool.com/blog/guide-qualitative-content-analysis)
* [How to Analyze Qualitative Data from UX Research: Thematic Analysis](https://www.nngroup.com/articles/thematic-analysis)
* [How to do a content analysis](https://paperpile.com/g/content-analysis/#what-is-content-analysis)

# Presentation Overview

* **[Data Preprocessing](https://en.wikipedia.org/wiki/Data_preprocessing)**: Clean data and ensure its properly standardized.
* **[N-Grams](https://en.wikipedia.org/wiki/N-gram)**: Provides quantitative data on word patterns and frequencies.
* **[Parts-of-Speech](https://en.wikipedia.org/wiki/Part_of_speech) Analysis**: Analyzes grammatical elements and their frequency.
    * The results from these analyses can be used to identify broader themes and patterns within the comments.
* **[Sentiment Analysis](https://en.wikipedia.org/wiki/Sentiment_analysis)**: Quantifies emotional tones across comments.

* **Counting occurrences and creating [indicator/dummy variables](https://en.wikipedia.org/wiki/Dummy_variable_(statistics))**: This quantifies the presence of each theme in the comments.
* **Calculating a [correlation](https://en.wikipedia.org/wiki/Correlation) matrix**: This step analyzes the relationships between different themes to understand how they co-occur.

# Load Code

## Load Dependencies

In [ ]:
import re
import os
import pandas as pd
import string
import nltk
from nltk import ngrams, word_tokenize
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()
from autocorrect import Speller
from collections import Counter
import subprocess # Used to launch Excel
import openpyxl #Used by Pandas to open Excel files
# from textblob import TextBlob #Used 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Download NLTK resources if not already downloaded. This only has to be done once.
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

## Load Functions

### Load to dataframe

In [ ]:
def load_file_to_dataframe(file_path):
    # Extract the file extension
    file_extension = file_path.split('.')[-1].lower()

    # Check the file extension and open the file accordingly
    if file_extension == 'csv':
        df = pd.read_csv(file_path)
    elif file_extension in ['xls', 'xlsx']:
        df = pd.read_excel(file_path)
    elif file_extension == 'json':
        df = pd.read_json(file_path)
    elif file_extension in ['txt', 'tsv']:
        delimiter = '\t' if file_extension == 'tsv' else ','
        df = pd.read_csv(file_path, delimiter=delimiter)
    elif file_extension == 'parquet':
        df = pd.read_parquet(file_path)
    elif file_extension == 'hdf':
        df = pd.read_hdf(file_path)
    elif file_extension == 'feather':
        df = pd.read_feather(file_path)
    else:
        raise ValueError("Unsupported file type. Supported types are: CSV, Excel (xls, xlsx), JSON, TXT, TSV, Parquet, HDF, Feather")

    return df


### Open in Excel

In [ ]:
def open_csv_in_excel(file_path):
    # check_excel_path_1 = r"C:\Program Files (x86)\Microsoft Office\root\Office16\EXCEL.EXE"
    # check_excel_path_2 = r"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE"
    excel_paths = [r"C:\Program Files (x86)\Microsoft Office\root\Office16\EXCEL.EXE", r"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE"]
    
    excel_program_path = None

    for excel_path in excel_paths:

        # Check if EXCEL.EXE exists in check_excel_path_1
        if os.path.exists(excel_path):
            excel_program_path = excel_path

    # # Check if EXCEL.EXE exists in check_excel_path_2
    # if not excel_path and os.path.exists(check_excel_path_2):
    #     excel_path = check_excel_path_2

    if not excel_program_path:
        # Directories to search for EXCEL.EXE
        directories_to_search = [
            r"C:\Program Files",
            r"C:\Program Files (x86)",
            r"C:\\",
            # Add more directories to search if needed
        ]

        excel_program_path = find_excel_exe(directories_to_search)

    if excel_program_path:
        subprocess.Popen([excel_program_path, file_path])
    else:
        print("Sorry. Cannot open file directly. Excel cannot be found")

def find_excel_exe(directories):
    for directory in directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.lower() == 'excel.exe':
                    return os.path.join(root, file)
    return None

def open_in_excel(filename):
    if os.path.exists(filename):
        print(f"Report written to: {filename}")
        open_csv = input(f"Open file in Excel? Y/N")
        if open_csv.lower() == "y":
            open_csv_in_excel(filename)
    else:
        print("Error writing report.")


### Misc Support Functions

In [ ]:
# Load a 2-column csv file as a pandas dataframe and return a dictionary 

def load_csv_to_dict(file_path, key_column, value_column):
    """
    Load a CSV file and create a dictionary from specified key and value columns.

    Parameters:
    file_path (str): The path to the CSV file.
    key_column (str): The name of the column to use as keys in the dictionary.
    value_column (str): The name of the column to use as values in the dictionary.

    Returns:
    dict: A dictionary with keys from the key_column and values from the value_column.
    """
    df = pd.read_csv(file_path)
    
    # Convert NaN values to empty strings
    df[value_column] = df[value_column].apply(lambda x: '' if pd.isna(x) else str(x))
    
    return df.set_index(key_column)[value_column].to_dict()

def ensure_string(text, string_length_threshold):
    """
    Ensure the input is converted to a string representation.

    Parameters:
    text: The input to be checked and converted to a string if necessary.
    string_length_threshold: The minimum length of the string. If the length of the string is less than this threshold, return an empty string.

    Returns:
    str: The input converted to a string or an empty string if the length is less than the threshold or conversion fails.
    """
    if pd.isnull(text):
        return ""
    elif isinstance(text, str):
        text = text.strip()  # Strip whitespace from both ends
        if len(text) < string_length_threshold:
            return ""
        return text
    
    try:
        # Attempt to convert non-string types to string
        text = str(text).strip()
        if len(text) < string_length_threshold:
            return ""
        return text
    except (ValueError, TypeError):
        return ""
    
def clean_whitespace(text):
    # Remove leading and trailing whitespace
    cleaned_text = text.strip()
    
    # Replace multiple spaces with a single space
    cleaned_text = ' '.join(cleaned_text.split())
    
    return cleaned_text


def save_dataframe_with_incremented_filename(file_path):
    """
    Check if a filename already exists. If it does, it returns an incremented filename.

    Parameters:
    df (pd.DataFrame): The DataFrame to be saved.
    file_path (str): The initial file path for the CSV file.
    """
    base, extension = os.path.splitext(file_path)
    counter = 1

    # Check if the file already exists
    while os.path.exists(file_path):
        # Increment the file name
        file_path = f"{base}_{counter}{extension}"
        counter += 1

    return file_path

### Text Analysis Functions

#### Lists and Settings

In [ ]:
# Stopword lisit
external_stopword_list = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\stopwords_seo.csv"

# Contractions dictionary
external_contractions_dictionary = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\contractions_list.csv"

# External search and replace file
custom_search_and_replace_dictionary = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\custom search-and-replace dictionary.csv"

# Load Custom Dictionaries
external_stopword_list_dict = load_csv_to_dict(external_stopword_list, "stopword", "replace")
contractions_dict = load_csv_to_dict(external_contractions_dictionary, "expansion", "contraction")
custom_search_and_replace_dict = load_csv_to_dict(custom_search_and_replace_dictionary, "search_term", "replace_term")

In [ ]:
common_encoding_errors = {
    "&lt;": "<",        # Less-than sign
    "&gt;": ">",        # Greater-than sign
    "&amp;": "&",       # Ampersand
    "&quot;": "\"",     # Double quotation mark
    "&apos;": "'",      # Apostrophe
    "&nbsp;": " ",      # Non-breaking space
    "&copy;": "(c)",    # Copyright sign
    "&reg;": "(r)",     # Registered sign
    "&euro;": "EUR",    # Euro sign
    "&pound;": "GBP",   # Pound sign
    "&yen;": "Yen",     # Yen sign
    "&cent;": "c",      # Cent sign
    "&sect;": "§",      # Section sign
    "&para;": "¶",      # Pilcrow sign
    "&deg;": "°",       # Degree sign
    "&plusmn;": "±",    # Plus-minus sign
    "&micro;": "µ",     # Micro sign
    "&sup2;": "^2",     # Superscript two
    "&sup3;": "^3",     # Superscript three
    "&frac14;": "1/4",  # Fraction one quarter
    "&frac12;": "1/2",  # Fraction one half
    "&frac34;": "3/4",  # Fraction three quarters
    "&times;": "x",     # Multiplication sign
    "&divide;": "/",    # Division sign
    "&bull;": "•",      # Bullet
    "&ndash;": "-",     # En dash
    "&mdash;": "-",     # Em dash
    "&lsquo;": "'",     # Left single quotation mark
    "&rsquo;": "'",     # Right single quotation mark
    "&sbquo;": ",",     # Single low-9 quotation mark
    "&ldquo;": "\"",    # Left double quotation mark
    "&rdquo;": "\"",    # Right double quotation mark
    "&bdquo;": "\"",    # Double low-9 quotation mark
    "&hellip;": "...",  # Horizontal ellipsis
    "%20": " ",         # Space
    "%21": "!",         # Exclamation mark
    "%22": "\"",        # Double quotation mark
    "%23": "#",         # Number sign
    "%24": "$",         # Dollar sign
    "%25": "%",         # Percent sign
    "%26": "&",         # Ampersand
    "%27": "'",         # Apostrophe
    "%28": "(",         # Left parenthesis
    "%29": ")",         # Right parenthesis
    "%2A": "*",         # Asterisk
    "%2B": "+",         # Plus sign
    "%2C": ",",         # Comma
    "%2D": "-",         # Hyphen
    "%2E": ".",         # Period
    "%2F": "/",         # Slash
    "%3A": ":",         # Colon
    "%3B": ";",         # Semicolon
    "%3C": "<",         # Less-than sign
    "%3D": "=",         # Equal sign
    "%3E": ">",         # Greater-than sign
    "%3F": "?",         # Question mark
    "%40": "@",         # At sign
    "%5B": "[",         # Left square bracket
    "%5C": "\\",        # Backslash
    "%5D": "]",         # Right square bracket
    "%5E": "^",         # Caret
    "%5F": "_",         # Underscore
    "%60": "`",         # Grave accent
    "%7B": "{",         # Left curly bracket
    "%7C": "|",         # Vertical bar
    "%7D": "}",         # Right curly bracket
    "%7E": "~",         # Tilde
    "â€”": "-",         # Em dash
    "â€“": "–",         # En dash
    "â€˜": "'",         # Left single quotation mark
    "â€™": "'",         # Right single quotation mark
    "â€œ": "\"",        # Left double quotation mark
    "â€": "\"",         # Right double quotation mark
    "â€¦": "...",       # Ellipsis
    "â€": "\"",         # Right double quotation mark (alternate)
    "â€™": "'",         # Right single quotation mark (alternate)
}

#### Text Cleaning Functions

In [ ]:
# Main Text cleaning function
def clean_text(text):
    global text_cleaning_options

    # Set Cleaning Options
    transform_contractions = text_cleaning_options["Transform contractions"]        
    convert_to_lowercase = text_cleaning_options["Convert to lowercase"]
    covert_common_encoding_errors = text_cleaning_options["Convert Common Encoding Errors"]
    
    strip_punctuation = text_cleaning_options["Strip punctuation"]
    remove_stopwords = text_cleaning_options["Remove stopwords"]

    use_custom_sar_dict = text_cleaning_options["Use custom Search-and-Replace dictionary"]
    standardize_highway_references = text_cleaning_options["Standardize highway references"]
    standardize_drivebc_references  = text_cleaning_options["Standardize drivebc references"]
    spell_check = text_cleaning_options["Spell check"]
    stem_words  = text_cleaning_options["Stem words"]
    lemmatize_words = text_cleaning_options["Lemmatize words"]

    try:
        # Make sure the data is actually text
        text = ensure_string(text,3)

        # Replace contractions
        if transform_contractions == "Expand":
            # Expand contractions
            for key, value in contractions_dict.items():
                text = text.replace(value, key)
            # Contract expansions
        elif transform_contractions == "Contract":
                for key, value in contractions_dict.items():
                    text = text.replace(key, value)

        if covert_common_encoding_errors is True:
            # Perform character replacement using the dictionary
            for improper, proper in common_encoding_errors.items():
                text = text.replace(improper, proper)
        
        # Convert all text to lower case
        if convert_to_lowercase is True:
            text = text.lower()

        
        # Standardize Highway References
        if standardize_highway_references is True:
            regex_highway_search1 = re.compile(
                r"#*\b(bc *)*h[ighway]*[\s#]*(\d+[abcd]*)\b",
                re.IGNORECASE,
            )
            regex_highway_replace1 = r"BCHwy\2"
            text = re.sub(regex_highway_search1, regex_highway_replace1, text)

            regex_highway_search2 = re.compile(
                r"\b(bc *)*h[ighway]*[\s#]*\b",
                re.IGNORECASE,
            )
            regex_highway_replace2 = r"highway"
            text = re.sub(regex_highway_search2, regex_highway_replace2, text)

        # Standardize DriveBC references
        if standardize_drivebc_references is True:
            regex_drivebc_search = re.compile(
                r"\bdrive.*bc\b",
                re.IGNORECASE,
            )
            regex_drivebc_replace = r"drivebc"
            text = re.sub(regex_drivebc_search, regex_drivebc_replace, text)

        # Use a custom search and replace dictionary
        if use_custom_sar_dict is True:
            for key, value in custom_search_and_replace_dict.items():
                text = text.replace(key, value)

        # Remove stop words
        if remove_stopwords is True:
            for key, value in external_stopword_list_dict.items():
                text = text.replace(key, value)

        # strip punctuation
        if strip_punctuation is True:
            # # This removed punctuation marks
            # text = " ".join([char for char in text if char not in string.punctuation])
            
            # This replaces punctuation marks with a space
            translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
            # Replace punctuation with spaces
            text = text.translate(translation_table)


        if spell_check is True:
            text = text.split()
            text = [spell(word) for word in text]
            text = " ".join(text)

        # Stem words
        if stem_words is True:
            text = text.split()
            text = [ps.stem(word) for word in text]
            text = " ".join(text)

        # Lemmatize words
        if lemmatize_words is True:
            text = text.split()
            text = [wn.lemmatize(word) for word in text]
            text = " ".join(text)

        # Clean Whitespace
        text = clean_whitespace(text)

        return text
    except Exception as e:
        print(f"Error processing text: {e}")
        return ""


# Add clean text to dataframe
def add_clean_text_column(df,comments):
    # print(df)
    df["clean_text"] = comments.apply(lambda x: clean_text(x))
    return df

### N-Gram Counter

In [ ]:
def ngram_counter(series):
    """
    Counts and returns the most common n-grams in a pandas Series of text data.

    The function prompts the user to input the starting and ending n-gram lengths, 
    as well as the number of top n-grams to find for each length. It then processes 
    the text data, tokenizes it, generates n-grams for the specified lengths, and 
    counts their occurrences. Finally, it returns a DataFrame with the n-grams, 
    their types, and counts.

    Parameters:
    series (pandas.Series): A pandas Series containing text data.

    Returns:
    pandas.DataFrame: A DataFrame with columns 'N-Gram', 'N-Gram Type', and 'Count',
                      containing the most common n-grams and their counts for each
                      specified n-gram length.

    User Inputs:
    - Starting n-gram length: An integer specifying the smallest n-gram length (e.g., 
                              1 for monogram, 2 for bigram, etc.).
    - Ending n-gram length: An integer specifying the largest n-gram length.
    - Number of n-grams: An integer specifying the number of top n-grams to find 
                         for each n-gram length.

    Example:
    >>> import pandas as pd
    >>> from nltk.tokenize import word_tokenize
    >>> from nltk.util import ngrams
    >>> from collections import Counter
    >>> series = pd.Series(["This is a sample text.", "This text is for testing purposes."])
    >>> result = ngram_counter(series)
    >>> print(result)
               N-Gram N-Gram Type  Count
    0            this       1-gram      2
    1              is       1-gram      2
    2              a       1-gram      1
    3          sample       1-gram      1
    4            text       1-gram      2
    5           this is     2-gram      2
    6              is a     2-gram      1
    7          sample text  2-gram      1
    8            text is    2-gram      1
    9              is for   2-gram      1
    10           for testing 2-gram     1
    11 testing purposes 2-gram  1
    """
    # Prompt user for the starting and ending n-gram lengths
    os.system("cls")
    while True:
        try:
            start_length = int(input("Enter the starting n-gram length (1 for monogram, 2 for bigram, etc.): "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")
    
    while True:
        try:
            end_length = int(input("Enter the ending n-gram length: "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    # Prompt user for the number of n-grams to find
    while True:
        try:
            num_ngrams = int(input("Enter the number of n-grams to find for each type: "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    ngrams_list = []

    # Process each n-gram length
    for n in range(start_length, end_length + 1):
        all_tokens = []

        # Tokenize each row and filter out rows with fewer words than the n-gram length
        for text in series.dropna().str.lower():
            tokens = word_tokenize(text)
            if len(tokens) >= n:
                all_tokens.extend(tokens)
        
        print(f"Tokens for n={n}:", all_tokens)  # Debug: Print tokens

        # Create n-grams
        generated_ngrams = list(ngrams(all_tokens, n))
        print(f"Generated {n}-grams:", generated_ngrams)  # Debug: Print generated n-grams

        ngram_counts = Counter(generated_ngrams)
        print(f"N-gram counts for n={n}:", ngram_counts)  # Debug: Print n-gram counts

        # Get the top n-grams of each type
        top_ngrams = ngram_counts.most_common(num_ngrams)
        for ngram, count in top_ngrams:
            ngrams_list.append([' '.join(ngram), f"{n}-gram", count])

    # Create a DataFrame with the n-grams, their types, and counts
    result_df = pd.DataFrame(ngrams_list, columns=['N-Gram', 'N-Gram Type', 'Count'])

    return result_df

### Isolate Parts of Speech Functions

In [ ]:
def isolate_pos(series, selected_parts_of_speech, string_length_threshold):
    if not selected_parts_of_speech:
        print("No Parts of Speech have been selected. Please select them and try again.")
        input(f"Press 'Enter' to continue.")
        return

    if isinstance(series, pd.Series):
        series = series.to_string()

    # Tokenize the sentence into individual words
    tokens = nltk.word_tokenize(series)

    # Use NLTK's POS tagger to tag each word with its part of speech
    pos_tags = nltk.pos_tag(tokens)

    # Create a list of words filtered by chosen Part-of-Speech (PoS) type
    word_list = []
    print("Tagging parts of speech...")
    for word, pos in pos_tags:
        if pos in selected_parts_of_speech:
            word_list.append(word)

    # Remove any stop words from the list
    stop_words = [
        "..",
        "'m",
        "'s",
        "#name?",
        "are",
        "be",
        "been",
        "being",
        "did",
        "do",
        "does",
        "done",
        "get",
        "go",
        "had",
        "has",
        "have",
        "i",
        "ì",
        "is",
        "make",
        "n/a",
        "needs",
        "put",
        "Q",
        "seems",
        "take",
        "use",
        "vs",
        "was",
        "way",
        "were",
    ]
    cleaned_word_list = [element for element in word_list if element not in stop_words]

    # Apply string length threshold
    filtered_word_list = [word for word in cleaned_word_list if len(word) >= string_length_threshold]

    # Count the frequency of each word
    word_counts = pd.Series(filtered_word_list).value_counts().reset_index()

    # Rename the columns of the DataFrame
    word_counts.columns = ["word", "count"]

    # Sort the DataFrame by count in descending order
    word_counts = word_counts.sort_values(by="count", ascending=False).reset_index(drop=True)

    return word_counts

### Sentiment Analysis

In [ ]:
def add_sentiment_column(df, text_column):
    """
    Add a sentiment score column to the DataFrame based on the specified text column using VADER.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    text_column (str): The name of the column containing the text for sentiment analysis.

    Returns:
    pd.DataFrame: The DataFrame with an added "sentiment" score column.
    """
    # Ensure the text_column exists in the DataFrame
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in DataFrame columns.")

    # Initialize the VADER sentiment intensity analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Define a function to calculate the sentiment score
    def get_sentiment(text):
        if pd.isnull(text) or not isinstance(text, str) or text.strip() == "":
            return None  # Return None for rows without valid text
        sentiment = analyzer.polarity_scores(text)
        return sentiment['compound']  # Compound sentiment score

    # Apply the sentiment function to the text column and add the results to a new column
    df['sentiment'] = df[text_column].apply(get_sentiment)

    return df

### Thematic Analysis

In [ ]:
def process_themes_and_comments(theme_list_path, comments_df, comments_column):
    # Load the theme list CSV file into a DataFrame
    theme_list_df = pd.read_csv(theme_list_path)

    # Create a dictionary to map terms to their associated themes
    term_to_themes = {}
    for _, row in theme_list_df.iterrows():
        terms = row['term'].split(',')
        themes = row['theme'].split(',')
        for term in terms:
            term = term.strip()
            if term not in term_to_themes:
                term_to_themes[term] = set()
            for theme in themes:
                term_to_themes[term].add(theme.strip())

    # Create new columns in comments_df for each unique theme
    unique_themes = set()
    for themes in term_to_themes.values():
        unique_themes.update(themes)

    for theme in unique_themes:
        comments_df[theme] = 0  # Initialize all theme columns with 0

    # Define a function to process each comment
    def process_comment(comment):
        if pd.isnull(comment):
            return {theme: 0 for theme in unique_themes}
        comment = comment.lower()
        result = {theme: 0 for theme in unique_themes}
        for term, themes in term_to_themes.items():
            if term.lower() in comment:
                for theme in themes:
                    result[theme] = 1
        return result

    # Check if comments_column exists
    if comments_column not in comments_df.columns:
        raise KeyError(f"Column '{comments_column}' not found in comments_df")

    # Apply the function to each row in the comments column
    results = comments_df[comments_column].apply(process_comment)

    # Update the comments_df with the results
    for theme in unique_themes:
        comments_df[theme] = results.apply(lambda x: x.get(theme, 0))

    return comments_df



### Correlation Analysis

In [ ]:
def analyze_correlation(df, threshold):
    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    
    if len(numeric_cols) < 2:
        raise ValueError("Not enough numeric columns to compute correlations.")
    
    # Compute the correlation matrix
    corr_matrix = df[numeric_cols].corr()
    
    # Apply the threshold to the correlation matrix
    filtered_corr_matrix = corr_matrix.applymap(lambda x: x if abs(x) >= threshold else np.nan)
    
    # Create a heatmap of the filtered correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(filtered_corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, mask=filtered_corr_matrix.isnull())
    plt.title('Correlation Matrix Heatmap (Filtered by Threshold)')
    plt.show()
    
    # Create a list of column pairs with their correlation scores
    corr_pairs = []
    for i in range(len(numeric_cols)):
        for j in range(i+1, len(numeric_cols)):
            col1 = numeric_cols[i]
            col2 = numeric_cols[j]
            corr_score = corr_matrix.loc[col1, col2]
            if abs(corr_score) >= threshold:
                corr_pairs.append(((col1, col2), corr_score))
    
    # Sort the list of column pairs by correlation score in descending order
    corr_pairs_sorted = sorted(corr_pairs, key=lambda x: abs(x[1]), reverse=True)
    
    return corr_pairs_sorted


### Latent Dirichlet Allocation (LDA) - Not used in this notebook. Requires additional packages. Kept here for reference.

In [75]:
def lda_analysis(series, num_topics=3, passes=10):
    """
    Perform LDA analysis on a pandas Series of text data.

    Parameters:
    series (pd.Series): A pandas Series containing text data.
    num_topics (int): The number of topics to extract. Default is 3.
    passes (int): The number of passes through the corpus during training. Default is 10.

    Returns:
    dict: A dictionary containing the LDA model, the document-topic matrix, 
          and the topic-term matrix for visualization.
    """
    # Define stopwords
    stop_words = set(stopwords.words('english'))

    # Preprocess text: tokenize, remove stopwords, punctuation, and non-alphabetic tokens
    def preprocess(text):
        tokens = word_tokenize(text.lower())
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
        return tokens

    # Apply preprocessing to the series
    processed_docs = series.dropna().apply(preprocess)

    # Create a dictionary and a corpus
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    # Train the LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

    # Assign topics to documents
    document_topics = lda_model.get_document_topics(corpus)
    document_topic_df = pd.DataFrame([[dict(doc)] for doc in document_topics])
    document_topic_df.columns = [f'Topic_{i + 1}' for i in range(num_topics)]
    document_topic_df['Document'] = series.reset_index(drop=True)

    # Prepare the visualization
    lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)

    # Collect results in a dictionary
    results = {
        "lda_model": lda_model,
        "document_topic_df": document_topic_df,
        "lda_vis": lda_vis
    }

    return results

# # Example usage:
# series = pd.Series([
#     "This is the first document.",
#     "This document is the second document.",
#     "And this is the third one.",
#     "Is this the first document?"
# ])

# # Perform LDA analysis
# results = lda_analysis(series, num_topics=3, passes=10)

# # Access the results
# lda_model = results['lda_model']
# document_topic_df = results['document_topic_df']
# lda_vis = results['lda_vis']

# # Print the topics
# for idx, topic in lda_model.print_topics(-1):
#     print(f"Topic {idx + 1}: {topic}")

# # Display the visualization in a Jupyter notebook (uncomment if using Jupyter)
# # pyLDAvis.display(lda_vis)

# Demo

## Load CSV Comments to DataFrame

In [ ]:
df = load_file_to_dataframe(r"D:\Data Analysis\Qualitative Analysis\comments_for_demo.csv")
df

## Select the Comments Column From DataFrame

In [ ]:
# define series
comments = df["Comment"]
comments

## N-Grams - First Pass (Noisy)

In [ ]:
# NGrams - Uncleaned
base_ngram_report = r"D:\Data Analysis\Qualitative Analysis\reports\ngram_report.csv"
ngram_report = save_dataframe_with_incremented_filename(base_ngram_report)
return_ngrams = ngram_counter(comments)
return_ngrams.to_csv(ngram_report,index=False)
open_in_excel(ngram_report)

## Text Cleaning

In [ ]:
text_cleaning_options = {
    "Convert to lowercase": True,
    "Convert Common Encoding Errors": True,
    "Strip punctuation": True,
    "Remove stopwords": False,
    "Transform contractions": "Expand", 
    "Use custom Search-and-Replace dictionary": True,
    "Standardize highway references": True,
    "Standardize drivebc references": True,
    "Spell check": False,
    "Stem words": False,
    "Lemmatize words": False,
}
cleaned_text = add_clean_text_column(df, comments)
cleaned_comments = cleaned_text["clean_text"]

base_cleaned_text_report = r"D:\Data Analysis\Qualitative Analysis\reports\cleaned_text_report.csv"
cleaned_text_report = save_dataframe_with_incremented_filename(base_cleaned_text_report)

cleaned_text.to_csv(cleaned_text_report,index=False)
open_in_excel(cleaned_text_report)

## N-Grams - Second Pass

In [ ]:
# NGrams - Cleaned
# print(type(comments))
# print(type(cleaned_comments))
base_ngram_report_cleaned = r"D:\Data Analysis\Qualitative Analysis\reports\ngram_report_cleaned.csv"
return_ngrams = ngram_counter(cleaned_comments)
ngram_report = save_dataframe_with_incremented_filename(base_ngram_report_cleaned)
return_ngrams.to_csv(ngram_report,index=False)
open_in_excel(ngram_report)

## Parts of Speech

In [ ]:
all_verbs = ["VB","VBD","VBG","VBN","VBP","VBZ"]
all_nouns = ["NN","NNP","NNS"]
all_nouns_and_verbs = ["NN","NNP","NNS","VB","VBD","VBG","VBN","VBP","VBZ"]

parts_of_speech = [
    ("CC", "conjunction, coordinating"),
    ("CD", "numeral, cardinal"),
    ("DT", "determiner"),
    ("EX", "existential there"),
    ("IN", "preposition or conjunction, subordinating"),
    ("JJ", "adjective or numeral, ordinal"),
    ("JJR", "adjective, comparative"),
    ("JJS", "adjective, superlative"),
    ("LS", "list item marker"),
    ("MD", "modal auxiliary"),
    ("NN", "noun, common, singular or mass"),
    ("NNP", "noun, proper, singular"),
    ("NNS", "noun, common, plural"),
    ("PDT", "pre-determiner"),
    ("POS", "genitive marker"),
    ("PRP", "pronoun, personal"),
    ("RB", "adverb"),
    ("RBR", "adverb, comparative"),
    ("RBS", "adverb, superlative"),
    ("RP", "particle"),
    ("TO", "to as preposition or infinitive marker"),
    ("UH", "interjection"),
    ("VB", "verb, base form"),
    ("VBD", "verb, past tense"),
    ("VBG", "verb, present participle or gerund"),
    ("VBN", "verb, past participle"),
    ("VBP", "verb, present tense, not 3rd person singular"),
    ("VBZ", "verb, present tense, 3rd person singular"),
    ("WDT", "WH-determiner"),
    ("WP", "WH-pronoun"),
    ("WRB", "Wh-adverb")
]

In [ ]:
pos_df = isolate_pos(cleaned_comments, all_nouns_and_verbs, 3)

# Specify the length threshold
string_length_threshold = 3

# Filter out rows where the length of the string in the "word" column is less than the threshold
filtered_pos_df = pos_df[pos_df['word'].apply(lambda x: len(x) >= string_length_threshold)]

base_pos_text_report = r"D:\Data Analysis\Qualitative Analysis\reports\parts_of_speech_text_report.csv"
pos_text_report = save_dataframe_with_incremented_filename(base_pos_text_report)

filtered_pos_df.to_csv(pos_text_report,index=False)
open_in_excel(pos_text_report)   

## Sentiment Analysis

In [ ]:
# Add sentiment column to the DataFrame
df_with_sentiment = add_sentiment_column(cleaned_text, 'Comment')

base_sentiment_report = r"D:\Data Analysis\Qualitative Analysis\reports\sentiment_report.csv"
sentiment_report = save_dataframe_with_incremented_filename(base_sentiment_report)

df_with_sentiment.to_csv(sentiment_report,index=False)
open_in_excel(sentiment_report)

## Thematic Analysis

In [ ]:
theme_list_path = r"D:\Data Analysis\Qualitative Analysis\Cleaning Text\lists and dictionaries\theme_list.csv"
comments_df = df_with_sentiment
comments_column = "clean_text"
theme_df = process_themes_and_comments(theme_list_path, comments_df, comments_column)

base_thematic_analysis_report = r"D:\Data Analysis\Qualitative Analysis\reports\thematic_analysis_report.csv"
thematic_analysis_report = save_dataframe_with_incremented_filename(base_thematic_analysis_report)

theme_df.to_csv(thematic_analysis_report,index=False)
open_in_excel(thematic_analysis_report)

## Correlation Analysis

In [ ]:
correlation_df = pd.read_csv(r"D:\Data Analysis\Qualitative Analysis\reports\thematic_analysis_report.csv")
threshold = 0.1
correlations = analyze_correlation(df, threshold)
for (col1, col2), score in correlations:
    print(f"{col1} and {col2}: {score:.2f}")